### Elasticsearch 文档创建与管理
本笔记本演示如何使用 Python 创建和管理 Elasticsearch 文档


In [1]:
from elasticsearch_dsl import connections,Index,Document,Text,Integer,Boolean,Keyword,Date

### 连接与索引设置
建立 Elasticsearch 连接并定义文档映射结构，包含字段类型和索引配置。


In [2]:
connection = connections.create_connection(hosts="http://localhost:9200");

In [3]:
class Books(Document):
    title = Text()
    description = Text()
    category = Keyword()
    price = Integer()
    in_stock = Boolean()
    timestamp = Date()  # 关键字段

    class Index:
        name="books"
        settings = {
            "number_of_shards": 1,
            "number_of_replicas": 0  # 开发环境建议设为0
        }
    # class Index:
    #     name="books";

In [4]:
if Index("books").exists():
    Index("books").delete

### 数据准备
从 CSV 文件加载书籍数据，包含标题、描述、类别、价格和库存状态等信息。


In [5]:
import pandas as pd

df_book = pd.read_csv("./data/books.csv")
df_book

,title,description,category,price,in_stock
0,西游记,由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描...,小说,10,False
1,红楼梦,由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。小说描绘了贾宝玉、林黛玉等人的生活，以及...,小说,20,True
2,水浒传,由明朝作家施耐庵创作，讲述了宋朝末年一群落难英雄在梁山泊起义反抗统治者的故事。小说通过塑造众...,小说,45,False
3,三国演义,由明朝作家罗贯中创作，讲述了三国时期的历史故事。小说通过描绘曹操、刘备、孙权等历史人物的生活...,小说,30,True
4,创意者的突破,"（""The Innovators: How a Group of Hackers, Geni...",传记,10,False
5,追风筝的人,"（""The Kite Runner""）：由卡勒德·胡赛尼（Khaled Hosseini）所...",传记,20,True
6,马拉拉的故事,"（""I Am Malala: The Girl Who Stood Up for Educa...",传记,45,False
7,爱因斯坦传,"（""Einstein: His Life and Universe""）：由沃尔特·艾萨克森所...",传记,35,True
8,Python,Python elastic search,工具,47,True
9,Term,Term-context,test,5,False


In [6]:
#构建文档

data = df_book.iloc[0,:]
print(data)
print(df_book[df_book.title.duplicated()])  # 检查DataFrame是否有重复

title                                                        西游记
description    由明朝作家吴承恩创作，讲述了孙悟空、猪八戒、沙僧等人与唐僧师徒四人西天取经的故事。小说通过描...
category                                                      小说
price                                                         10
in_stock                                                   False
Name: 0, dtype: object
Empty DataFrame
Columns: [title, description, category, price, in_stock]
Index: []


### 单文档创建
演示如何创建单个文档并保存到 Elasticsearch 索引中。


In [7]:
book = Books(
    title = data.title,description=data.description,
    category=data.category,price=data.price,in_stock=data.in_stock
)
book

Books()

### 批量文档创建
使用循环批量导入所有书籍数据到 Elasticsearch 索引。


In [8]:
from datetime import datetime


for row_no,row in df_book.iterrows():
    print(row_no)
    book = Books(
        title = row.title,description=row.description,
        category=row.category,price=row.price,in_stock=row.in_stock,
        #timestamp=datetime.utcnow()  # 自动添加当前时间
    )
    book.save()

0
1
2
3
4
5
6
7
8
9


### 数据验证
检查数据重复性，确保导入数据的完整性。


In [9]:
print(df_book[df_book.title.duplicated()])  # 检查DataFrame是否有重复

Empty DataFrame
Columns: [title, description, category, price, in_stock]
Index: []
